In [1]:
import os

os.chdir('../scripts') # cd into scripts dir

from scan import *
from prep import *
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression

os.chdir('../') # cd back to parent dir
home_dir = "compas/" # set home dir for reading and storing data

# Read recipe inputs
compas_prep_df = pd.read_csv(home_dir + "datasets/compas_prep.csv")
compas_prep_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,compas_risk_score,proba_compas,proba_lr
0,0,Male,Other,None,False,F,0,1,0.213889,0.232134
1,1,Male,African-American,None,False,F,1,3,0.376171,0.289774
2,2,Male,African-American,1 to 5,True,F,1,4,0.434330,0.668642
3,3,Male,African-American,1 to 5,True,F,0,8,0.683594,0.668642
4,4,Male,Other,1 to 5,False,F,0,1,0.213889,0.373653
...,...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,7,0.591216,0.505581
7210,7210,Male,African-American,None,True,F,0,3,0.376171,0.505581
7211,7211,Male,Other,None,False,F,0,1,0.213889,0.232134
7212,7212,Female,African-American,1 to 5,False,M,0,2,0.311371,0.335035


### Prepare Base Dataset for Simulations

Select only covariates and outcomes to prepare for simulation.

In [2]:
# Exclude risk score
prob_cols = ['compas_risk_score', 'proba_lr']
compas_sim_df = compas_prep_df.drop(columns=prob_cols)

### Build Logistic Regression Model to Predict Outcomes

One-Hot encode categorical features.

In [3]:
### Columns to exclude from model
exclude = ['race', 'under_25', 'Unnamed: 0']

In [4]:
compas_df = compas_sim_df.copy(deep=True)
compas_df = compas_df.drop(columns=exclude)
compas_df

,sex,prior_offenses,charge_degree,outcomes,proba_compas
0,Male,None,F,0,0.213889
1,Male,None,F,1,0.376171
2,Male,1 to 5,F,1,0.434330
3,Male,1 to 5,F,0,0.683594
4,Male,1 to 5,F,0,0.213889
...,...,...,...,...,...
7209,Male,None,F,0,0.591216
7210,Male,None,F,0,0.376171
7211,Male,None,F,0,0.213889
7212,Female,1 to 5,M,0,0.311371


In [5]:
for cat_col in compas_df.select_dtypes(include=['object', 'bool']).columns:
    compas_df[cat_col] = compas_df[cat_col].astype('category')

In [6]:
X_num = compas_df.select_dtypes(exclude=['category'])
X_cat = compas_df.select_dtypes(include=['category'])

In [7]:
X_cat

,sex,prior_offenses,charge_degree
0,Male,None,F
1,Male,None,F
2,Male,1 to 5,F
3,Male,1 to 5,F
4,Male,1 to 5,F
...,...,...,...
7209,Male,None,F
7210,Male,None,F
7211,Male,None,F
7212,Female,1 to 5,M


In [8]:
# one-hot encoding of categorical features
X_encoded = pd.get_dummies(X_cat)
frames = [X_encoded, X_num]
compas_df = pd.concat(frames, axis=1)

In [9]:
extra_cols = ['sex_Female', 'prior_offenses_None', 'charge_degree_M']
compas_df = compas_df.drop(columns=extra_cols)
compas_df

,sex_Male,prior_offenses_1 to 5,prior_offenses_Over 5,charge_degree_F,outcomes,proba_compas
0,1,0,0,1,0,0.213889
1,1,0,0,1,1,0.376171
2,1,1,0,1,1,0.434330
3,1,1,0,1,0,0.683594
4,1,1,0,1,0,0.213889
...,...,...,...,...,...,...
7209,1,0,0,1,0,0.591216
7210,1,0,0,1,0,0.376171
7211,1,0,0,1,0,0.213889
7212,0,1,0,0,0,0.311371


In [10]:
X = compas_df.drop(columns=['outcomes'])
y = compas_df['outcomes']

In [11]:
lr = LogisticRegression(random_state=0, solver='lbfgs')
lr.fit(X, y)
proba = lr.predict_proba(X)[:,1]
compas_sim_df['proba'] = proba
compas_sim_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,proba_compas,proba
0,0,Male,Other,None,False,F,0,0.213889,0.206231
1,1,Male,African-American,None,False,F,1,0.376171,0.313529
2,2,Male,African-American,1 to 5,True,F,1,0.434330,0.460612
3,3,Male,African-American,1 to 5,True,F,0,0.683594,0.670093
4,4,Male,Other,1 to 5,False,F,0,0.213889,0.284108
...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,0.591216,0.490967
7210,7210,Male,African-American,None,True,F,0,0.376171,0.313529
7211,7211,Male,Other,None,False,F,0,0.213889,0.206231
7212,7212,Female,African-American,1 to 5,False,M,0,0.311371,0.259957


In [12]:
# Check accuracy
# Note: model results can vary environment to environment, which can impact IJDI scores.
compas_sim_df['test_outcomes'] = compas_sim_df['proba'].apply(lambda x : 1 if x > 0.5 else 0)
compas_sim_df['check'] = compas_sim_df.apply(lambda x : 1 if x['outcomes'] == x['test_outcomes'] else 0, axis=1)
print(np.sum(compas_sim_df['check']) / len(compas_sim_df))
compas_sim_df = compas_sim_df.drop(columns=['test_outcomes', 'check'])

0.6639866925422789


### Example of one Iteration for Base Case of the Simulation 2

Define subgroup for which to simulate altered probabilities.

In [13]:
pd.options.mode.chained_assignment = None  # suppress warnings on chained assignment
subgroup = {'race':['African-American', 'Asian'], 'under_25':[True]}
compas_sim_df['in_subgroup'] = compas_sim_df[list(subgroup.keys())].isin(subgroup).all(axis=1)
compas_sim_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,proba_compas,proba,in_subgroup
0,0,Male,Other,None,False,F,0,0.213889,0.206231,False
1,1,Male,African-American,None,False,F,1,0.376171,0.313529,False
2,2,Male,African-American,1 to 5,True,F,1,0.434330,0.460612,True
3,3,Male,African-American,1 to 5,True,F,0,0.683594,0.670093,True
4,4,Male,Other,1 to 5,False,F,0,0.213889,0.284108,False
...,...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,0.591216,0.490967,True
7210,7210,Male,African-American,None,True,F,0,0.376171,0.313529,True
7211,7211,Male,Other,None,False,F,0,0.213889,0.206231,False
7212,7212,Female,African-American,1 to 5,False,M,0,0.311371,0.259957,False


In [14]:
THRESHOLD = 0.5
proba_epsilon = 0.3
threshold_epsilon = 0.3
compas_sim_df['proba_shifted'] = compas_sim_df.apply(lambda x : calculate_proba_shift(x['proba'], proba_epsilon) if x['in_subgroup']
                                                                                                                 else x['proba'], axis=1)
compas_sim_df['threshold'] = THRESHOLD
compas_sim_df['threshold_shifted'] = compas_sim_df.apply(lambda x : calculate_proba_shift(x['threshold'], -threshold_epsilon) if x['in_subgroup']
                                                                                                                              else x['threshold'], axis=1)
compas_sim_df['outcomes'] = compas_sim_df['proba'].apply(lambda x : generate_outcomes(x))
compas_sim_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,proba_compas,proba,in_subgroup,proba_shifted,threshold,threshold_shifted
0,0,Male,Other,None,False,F,1,0.213889,0.206231,False,0.206231,0.5,0.500000
1,1,Male,African-American,None,False,F,1,0.376171,0.313529,False,0.313529,0.5,0.500000
2,2,Male,African-American,1 to 5,True,F,1,0.434330,0.460612,True,0.535470,0.5,0.425557
3,3,Male,African-American,1 to 5,True,F,0,0.683594,0.670093,True,0.732747,0.5,0.425557
4,4,Male,Other,1 to 5,False,F,0,0.213889,0.284108,False,0.284108,0.5,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,0.591216,0.490967,True,0.565585,0.5,0.425557
7210,7210,Male,African-American,None,True,F,0,0.376171,0.313529,True,0.381385,0.5,0.425557
7211,7211,Male,Other,None,False,F,0,0.213889,0.206231,False,0.206231,0.5,0.500000
7212,7212,Female,African-American,1 to 5,False,M,0,0.311371,0.259957,False,0.259957,0.5,0.500000


Generate variant metrics for only "truly negative-outcome" people (those who did NOT reoffend). Additionally, generate metrics required for IJDI scan (i.e. $\hat{p}$ calculation).

In [15]:
# specify probability and outcomes columns
OUTCOMES_COL = 'outcomes'
FEATURES = ['sex', 'race', 'under_25', 'charge_degree']
LAMBDA_PARAM = 10
pd.options.mode.chained_assignment = None  # suppress warnings on chained assignment

# filter for only negative outcomes
negatives_df = compas_sim_df.loc[compas_sim_df[OUTCOMES_COL] == 0]

In [16]:
# # define new columns
# metrics_df = generate_metrics(negatives_df, 'proba_shifted', OUTCOMES_COL, THRESHOLD, constant_threshold=False)

# print("Number of people in subgroup:", len(negatives_df[negatives_df['in_subgroup'] == True]))
# print("Number of people not in subgroup:", len(negatives_df[negatives_df['in_subgroup'] == False]))
# print("Total population:", len(negatives_df))

# # display preview of treatments, probabilities, and outcomes
# metrics_df.head(10)

In [17]:
# # scan with shifted probabilities
# current_subset, current_score = run_ijdi_scan(negatives_df, FEATURES, 'proba', 'proba', OUTCOMES_COL,
#                                               'threshold_shifted', LAMBDA_PARAM, constant_threshold=False)
# summarize_scan(negatives_df, FEATURES, 'proba', OUTCOMES_COL, current_subset, include='all')

In [18]:
# # scan with shifted thresholds
# current_subset, current_score = run_ijdi_scan(negatives_df, FEATURES, 'proba_shifted', 'proba', OUTCOMES_COL,
#                                               'threshold', LAMBDA_PARAM, constant_threshold=False)
# summarize_scan(negatives_df, FEATURES, 'proba', OUTCOMES_COL, current_subset, include='all')

### Simulations for Negatives

Set mean values as well as ranges of standard deviation and lambda to iterate over for each simulation. Also define threshold and number of iterations to run for each set of parameters.

In [19]:
epsilon_vals = [0, 0.3, 3]
lambda_vals = [0, 0.3, 1, 3, 10]
n_iters = 10

# specify parameters for generating metrics and IJDI scan
outcomes_col = 'outcomes'
features = ['sex', 'race', 'under_25', 'charge_degree']
threshold = 0.5
pd.options.mode.chained_assignment = None  # suppress warnings on chained assignment

# set random seed
np.random.seed(300)

In [20]:
sim_data = []

for e in epsilon_vals:

    print("Parameter Epsilon =", e)

    for i in range(n_iters): # run n iterations for each k value

        print("Simulation", i+1, "of", n_iters)

        compas_sim_df['proba_shifted'] = compas_sim_df.apply(lambda x : calculate_proba_shift(x['proba'], e) if x['in_subgroup']
                                                                                                             else x['proba'], axis=1)
        compas_sim_df['threshold'] = threshold
        compas_sim_df['threshold_shifted'] = compas_sim_df.apply(lambda x : calculate_proba_shift(x['threshold'], -e) if x['in_subgroup']
                                                                                                                      else x['threshold'], axis=1)
        compas_sim_df['outcomes'] = compas_sim_df['proba'].apply(lambda x : generate_outcomes(x))

        # filter for only negative outcomes
        negatives_df = compas_sim_df.loc[compas_sim_df[outcomes_col] == 0]

        for lambda_param in lambda_vals: # run IJDI scan for various lambda values

            print("Lambda =", lambda_param)
            
            negatives_df_copy_1 = negatives_df.copy(deep=True)
            negatives_df_copy_2 = negatives_df.copy(deep=True)

            # Run IJDI Scan. Make sure to pass in copy because data may be modified by the function!
            proba_shift_subset, proba_shift_score = run_ijdi_scan(negatives_df_copy_1, features, 'proba_shifted', 'proba', outcomes_col,
                                                                  'threshold', lambda_param, constant_threshold=False, verbose=False)
            threshold_shift_subset, threshold_shift_score = run_ijdi_scan(negatives_df_copy_2, features, 'proba', 'proba', outcomes_col,
                                                                          'threshold_shifted', lambda_param, constant_threshold=False, verbose=False)

            # save iou and score
            if proba_shift_subset:
                in_subgroup = negatives_df[list(subgroup.keys())].isin(subgroup).all(axis=1)
                in_proba_shift_subset = negatives_df[list(proba_shift_subset.keys())].isin(proba_shift_subset).all(axis=1)
                proba_shift_iou = (in_subgroup & in_proba_shift_subset).sum() / (in_subgroup | in_proba_shift_subset).sum()
            else:
                proba_shift_iou = 0.0

            print("Detected proba shift subset intersection over union with expected subset:", proba_shift_iou)
            print("Detected proba shift subset score:", proba_shift_score)

            if threshold_shift_subset:
                in_subgroup = negatives_df[list(subgroup.keys())].isin(subgroup).all(axis=1)
                in_threshold_shift_subset = negatives_df[list(threshold_shift_subset.keys())].isin(threshold_shift_subset).all(axis=1)
                threshold_shift_iou = (in_subgroup & in_threshold_shift_subset).sum() / (in_subgroup | in_threshold_shift_subset).sum()
            else:
                threshold_shift_iou = 0.0

            print("Detected threshold shift subset intersection over union with expected subset:", threshold_shift_iou)
            print("Detected threshold shift subset score:", threshold_shift_score)

            # append data
            sim_row = [e, lambda_param, proba_shift_iou, proba_shift_score, threshold_shift_iou, threshold_shift_score]
            sim_data.append(sim_row)
            print(sim_row)
                        
            # release memory from kernel
            release_df(negatives_df_copy_1)
            release_df(negatives_df_copy_2)
            
            print("\n----------------------------------------------------\n")

Parameter Epsilon = 0
Simulation 1 of 10
Lambda = 0
Required time =  19.289342164993286 seconds
Found positive subset for ijdi scan:
{'race': ['African-American', 'Native American'], 'sex': ['Male']}
Score: 98.16566374058944
First Iteration
Average p_delta: 0.07954319744399185
Average p_censor: 0.0
p(S): 0.4656759313210879 p(~S): 0.3407956126924402 E[censored] 0.21957936905357767 E[uncensored] 0.21957936905357767
Subset does not violate p_delta or p_censor conditions!
Required time =  20.53980278968811 seconds
Found positive subset for ijdi scan:
{'sex': ['Male'], 'race': ['African-American', 'Native American']}
Score: 98.16566374058944
First Iteration
Average p_delta: 0.07954319744399185
Average p_censor: 0.0
p(S): 0.4656759313210879 p(~S): 0.3407956126924402 E[censored] 0.21957936905357767 E[uncensored] 0.21957936905357767
Subset does not violate p_delta or p_censor conditions!
Detected proba shift subset intersection over union with expected subset: 0.2257234726688103
Detected proba

In [21]:
columns = ['e', 'lambda', 'proba_shift_iou', 'proba_shift_score', 'threshold_shift_iou', 'threshold_shift_score']
sim_result_df = pd.DataFrame(sim_data, columns=columns)
sim_result_df

,e,lambda,proba_shift_iou,proba_shift_score,threshold_shift_iou,threshold_shift_score
0,0.0,0.0,0.225723,98.165664,0.225723,98.165664
1,0.0,0.3,0.225723,69.849854,0.225723,69.849854
2,0.0,1.0,0.225723,26.961449,0.225723,26.961449
3,0.0,3.0,0.000000,0.310828,0.000000,0.000000
4,0.0,10.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
145,3.0,0.0,1.000000,582.023406,1.000000,582.023406
146,3.0,0.3,1.000000,548.469878,1.000000,548.469878
147,3.0,1.0,1.000000,502.626959,1.000000,502.626959
148,3.0,3.0,1.000000,443.241668,1.000000,443.241668


In [22]:
# Write recipe outputs
sim_result_df.to_csv(home_dir + "datasets/compas_sim_2_neg.csv")